In [105]:
from tqdm import tqdm
import numpy as np
import pickle as pkl
from IPython.display import clear_output

In [49]:
import re

lc = 0

nodes = []
ways = []

with open('./data/map(3)', 'r') as f:
    
    for line in f:
        
        if re.match('\s+<node [^>]+>', line):
            
            if re.match('\s+<node [^>]+\/>', line):
                nodeItem = {i.split('=')[0]: i.split('=')[1].strip('"') for i in re.findall('\w+=\"[^\"]+\"', line)}
                nodeItem['lat'] = float(nodeItem['lat'])
                nodeItem['lon'] = float(nodeItem['lon'])
                
                nodes.append(nodeItem)
            else:
                nodeItem = {}
                
                nodeItem = {i.split('=')[0]: i.split('=')[1].strip('"') for i in re.findall('\w+=\"[^\"]+\"', line)}
                nodeItem['lat'] = float(nodeItem['lat'])
                nodeItem['lon'] = float(nodeItem['lon'])

                for nodeLine in f:

                    if re.match('\s+<\/node>', nodeLine):
                        
                        if 'addr:street' in nodeItem:
                            
                            nodeItem = {k: v for k, v in nodeItem.items() if (k in ['id', 'lat', 'lon'])or('addr' in k)}
                            
                            nodes.append(nodeItem)
                            
                        nodeItem = {}
                        break

                    m = re.match('\s+<tag k=\"(.+)\" v=\"(.+)\"\/>', nodeLine)
                    
                    if m is not None:
                        nodeItem[m[1]] = m[2]
                        
        if re.match('\s+<way [^>]+>', line):
            
            if re.match('\s+<way [^>]+\/>', line):
                pass
            else:
                nodeItem = {}
                
                nodeItem = {i.split('=')[0]: i.split('=')[1].strip('"') for i in re.findall('\w+=\"[^\"]+\"', line)}

                for nodeLine in f:

                    if re.match('\s+<\/way>', nodeLine):
                        
                        if 'addr:street' in nodeItem:
                            
                            nodeItem = {
                                k: v 
                                for k, v in nodeItem.items() if (k in ['id', 'lat', 'lon', 'refs'])or('addr' in k)
                            }
                            
                            ways.append(nodeItem)
                            
                        nodeItem = {}
                        break

                    m = re.match('\s+<tag k=\"(.+)\" v=\"(.+)\"\/>', nodeLine)
                    if m is not None:
                        nodeItem[m[1]] = m[2]
                        
                    m = re.match('\s+<nd ref=\"(.+)\"\/>', nodeLine)
                    if m is not None:
                        if 'ref' not in nodeItem:
                            nodeItem['refs'] = []
                        nodeItem['refs'].append(m[1])

In [50]:
import pymongo

db = pymongo.MongoClient()['ltcLongevity']

In [107]:
addresses = []

for user in db.usersV2.find({}):
    try:
        userId = user['userId']
        address = user['address'].split(',')
        city = address[0]
        street = address[1]
        home = address[2]
        
        street = street.replace('пер.', "переулок")
        street = street.replace('ул.', "улица")
        street = street.replace('пр-т', "проспект")
        street = street.replace('пр-кт', "проспект")
        street = street.replace('б-р', "бульвар")
        
        
        addresses.append({
            'userId': userId,
            'street': street.strip(),
            'home': home.strip()
        })
    except:
        pass

In [113]:
nodesMap = {
    int(item['id']): {'lat': item['lat'], 'lon': item['lon']}
    for item in nodes
}

In [116]:
def compareSets(s1, s2):
    
    for ss1 in s1:
        if ss1 not in s2:
            return False
    return True

def getCoordsForWay(waysFound):

    coords = []

    for w in waysFound:
        for r in w['refs']:
            if int(r) in nodesMap:
                coords.append(nodesMap[int(r)])
                break
                    
    coords = {
        'lat': np.mean([i['lat'] for i in coords]),
        'lon': np.mean([i['lon'] for i in coords])
    }

    return coords

goods = 0
bads = 0

coordsMap = {}

for addr in addresses:
    
    waysFound = []
    
    for w in ways:
        if addr['street'].replace('ё', "е") in w['addr:street'].lower().replace('ё', "е"):
            waysFound.append(w)
            
    if len(waysFound) == 0:
        
        for w in ways:
            if compareSets(addr['street'].replace('ё', "е").split(), w['addr:street'].lower().replace('ё', "е").split()):
                waysFound.append(w)
        
        if len(waysFound) == 0:
            bads += 1

    if len(waysFound) > 0:
        coordsMap[addr['userId']] = getCoordsForWay(waysFound)
        goods += 1
        
    clear_output(wait = True)
    print('Goods: %d, bads: %d, ratio: %4.2f%%'%(goods, bads, bads / (goods + bads) * 100))

Goods: 49629, bads: 2706, ratio: 5.17%


In [119]:
pkl.dump(coordsMap, open('./data/coordsMap.pkl', 'wb'))

In [120]:
coordsMap

{101391104: {'lat': 55.81696185, 'lon': 37.65629973125},
 101437383: {'lat': 55.72398774456522, 'lon': 37.62608760652174},
 101351338: {'lat': 55.62345309999999, 'lon': 37.30614648461538},
 101354578: {'lat': 55.68368901146496, 'lon': 37.48614224394905},
 101411997: {'lat': 55.70245170434782, 'lon': 37.942805986956515},
 101387414: {'lat': 55.65187056442308, 'lon': 37.605914048076926},
 101451078: {'lat': 55.64565921623377, 'lon': 37.51842641428572},
 101396634: {'lat': 55.72042862272727, 'lon': 37.6174757909091},
 101406313: {'lat': 55.711630135593204, 'lon': 37.779774538983055},
 101362471: {'lat': 55.77829957427185, 'lon': 37.584752041262135},
 101385694: {'lat': 55.637892674999996, 'lon': 37.66161538611111},
 101375942: {'lat': 55.62308929014084, 'lon': 37.65335273239437},
 101444070: {'lat': 55.691661575, 'lon': 37.8559713375},
 101440071: {'lat': 55.61514628512396, 'lon': 37.60451171239669},
 101377171: {'lat': 55.729721664705885, 'lon': 37.82627414764706},
 101383297: {'lat': 55